In [1]:
import polars as pl 

In [36]:
hifi = pl.read_csv("data_hifi_pre_release.index.csv")
hifi_bams = (
    hifi.filter(pl.col("filetype").eq("bam") & pl.col("MM_remove").eq(False))
    .select(["sample_ID", "path", "deepconsensus_path", "coverage", "deepconsensus_coverage", "library_ID", "instrument_model"])

)

hifi_bams_max_cov = (
    hifi_bams
    .filter(pl.col("coverage") >= pl.col("coverage").max().over("sample_ID"))
    .group_by(pl.col("sample_ID"))
    .agg(pl.all().first())
    .sort(["coverage", "deepconsensus_coverage"], descending=True)
    .rename(lambda column_name: "pacbio_" + column_name)
)
hifi_bams_max_cov

pacbio_sample_ID,pacbio_path,pacbio_deepconsensus_path,pacbio_coverage,pacbio_deepconsensus_coverage,pacbio_library_ID,pacbio_instrument_model
str,str,str,f64,f64,str,str
"""NA18974""","""s3://human-pangenomics/working…",null,42.0,null,"""NA18974_lib1_m84081_230601_220…","""Revio"""
"""NA21144""","""s3://human-pangenomics/working…",null,41.3,null,"""NA21144_lib1_m84081_230629_192…","""Revio"""
"""NA18608""","""s3://human-pangenomics/working…",null,40.7,null,"""NA18608_lib1_m84081_230601_210…","""Revio"""
"""HG00320""","""s3://human-pangenomics/working…",null,39.5,null,"""HG00320_lib1_m84081_230610_205…","""Revio"""
"""NA18952""","""s3://human-pangenomics/working…",null,39.3,null,"""NA18952_lib1_m84081_230609_204…","""Revio"""
…,…,…,…,…,…,…
"""HG01928""","""s3://human-pangenomics/working…","""s3://human-pangenomics/submiss…",9.2,10.85,"""HG01928_lib1""","""Sequel II"""
"""HG02486""","""s3://human-pangenomics/working…","""s3://human-pangenomics/submiss…",8.9,10.38,"""HG02486.HiFiEx2_f2""","""Sequel II"""
"""HG03195""","""s3://human-pangenomics/working…",null,7.0,null,"""HG03195_lib1""","""Revio"""


In [39]:
ont = (
    pl.read_csv("data_ont_pre_release.index.csv")
    .filter(pl.col("filetype").eq("bam"))
    .select(["sample_ID", "path", "coverage", "sequencing_chemistry", "instrument_model", "library_ID"])
    .filter(pl.col("coverage") >= pl.col("coverage").max().over("sample_ID"))
    .group_by(pl.col("sample_ID"))
    .agg(pl.all().first())
    .sort("coverage", descending=True)
    .rename(lambda column_name: "ont_" + column_name)

)
ont

ont_sample_ID,ont_path,ont_coverage,ont_sequencing_chemistry,ont_instrument_model,ont_library_ID
str,str,f64,str,str,str
"""HG00597""","""s3://human-pangenomics/working…",36.3,"""R941""","""PromethION""","""07_27_21_R941_HG00597"""
"""HG01993""","""s3://human-pangenomics/working…",35.76,"""R941""","""PromethION""","""08_10_21_R941_HG01993"""
"""HG02004""","""s3://human-pangenomics/working…",35.45,"""R941""","""PromethION""","""08_25_21_R941_HG02004"""
"""HG02630""","""s3://human-pangenomics/working…",34.99,"""R941""","""PromethION""","""04_11_23_R941_HG02630_2_Guppy_…"
"""HG00329""","""s3://human-pangenomics/working…",34.92,"""R1041""","""PromethION""","""10_24_23_R1041_HPRC_HG00329_3"""
…,…,…,…,…,…
"""HG02257""","""s3://human-pangenomics/working…",15.57,"""R941""","""PromethION""","""12_7_22_R941_HG02257_2_Guppy_6…"
"""HG03492""","""s3://human-pangenomics/working…",15.15,"""R941""","""PromethION""","""HG03492_1"""
"""HG00146""","""s3://human-pangenomics/working…",15.05,"""R1041""","""PromethION""","""01_23_24_R1041_HPRC_HG00146_1"""


In [38]:
illumina = (
    pl.read_csv("data_illumina_pre_release.index.csv", ignore_errors=True)
    .select(["sample_ID","path", "coverage", "instrument_model", 
             "Family ID", "Paternal ID", "Maternal ID", 
             "Relationship", "Gender", "Population", "phasing"])
    .filter(pl.col("coverage") >= pl.col("coverage").max().over("sample_ID"))
    .group_by(pl.col("sample_ID"))
    .agg(pl.all().first())
    .sort("coverage", descending=True)
    .rename(lambda column_name: "illumina_" + column_name)
)

illumina

illumina_sample_ID,illumina_path,illumina_coverage,illumina_instrument_model,illumina_Family ID,illumina_Paternal ID,illumina_Maternal ID,illumina_Relationship,illumina_Gender,illumina_Population,illumina_phasing
str,str,f64,str,str,str,str,str,i64,str,str
"""HG00232""","""s3://human-pangenomics/working…",61.31,"""Illumina NovaSeq 6000""","""HG00232""","""0""","""0""","""unrel""",2,"""GBR""","""hic"""
"""HG03270""","""s3://human-pangenomics/working…",61.15,"""Illumina NovaSeq 6000""","""NG65""","""0""","""0""","""mother""",2,"""ESN""","""hic"""
"""HG00321""","""s3://human-pangenomics/working…",60.18,"""Illumina NovaSeq 6000""","""HG00321""","""0""","""0""","""unrel""",1,"""FIN""","""hic"""
"""NA21106""","""s3://human-pangenomics/working…",57.83,"""Illumina NovaSeq 6000""","""NA21106""","""0""","""0""","""unrel""",2,"""GIH""","""hic"""
"""NA20762""","""s3://human-pangenomics/working…",57.48,"""Illumina NovaSeq 6000""","""NA20762""","""0""","""0""","""unrel""",1,"""TSI""","""hic"""
…,…,…,…,…,…,…,…,…,…,…
"""HG00639""","""s3://human-pangenomics/working…",33.32,"""Illumina NovaSeq 6000""","""PR03""","""HG00637""","""HG00638""","""child""",2,"""PUR""","""trio"""
"""HG00323""","""s3://human-pangenomics/working…",33.27,"""Illumina NovaSeq 6000""","""HG00323""","""0""","""0""","""unrel""",2,"""FIN""","""hic"""
"""NA18983""","""s3://human-pangenomics/working…",33.12,"""Illumina NovaSeq 6000""","""NA18983""","""0""","""0""","""unrel""",1,"""JPT""","""hic"""


In [51]:
hprc_overlapped_samples = (
    illumina.join(hifi_bams_max_cov, 
              how = "inner", 
              left_on="illumina_sample_ID", 
              right_on="pacbio_sample_ID")
            .join(ont, how="inner", left_on="illumina_sample_ID", right_on="ont_sample_ID")   
            .rename(lambda column_name: column_name.replace("illumina_", ""))         
)
hprc_overlapped_samples.write_csv("hprc_overlapped_samples_metadata.csv")


In [66]:
final_manifest = (
    hprc_overlapped_samples.select(
    ["sample_ID", "path", "ont_path", "pacbio_path"]
    )
    .rename({"path": "illumina_path"})
    .unpivot(index ="sample_ID", variable_name="type", value_name="path")
    .with_columns(
        type = pl.col("type").str.replace("_path", '')
    )
    .with_columns(
        align = pl.when(pl.col("type").is_in(["pacbio", "ont"])).then(pl.lit("yes")).otherwise(pl.lit("no"))
    )
    .sort("sample_ID")
)

final_manifest.write_csv("manifest.tsv", include_header=False, separator="\t")